In [1]:
# Import packages
import pandas as pd
import numpy as np
from unidecode import unidecode
from datetime import datetime
import ast

In [2]:
def replaceClubsName(series):
    clubs_name_replace_list = {'^1\. ':'','^RC ':'','^RCD ':'','^UD ':'',' UD$':'',' CF$':'','^CF ':'',' FC$':'',' Football Club':'','^AS ':'','^CD ':'',' SA$':'','^CA ':'','^ESTAC ':'',' AC$':'','^ES ':'','^Racing Club de ':'',' Atletico$':'','^OGC ':'',' FCO$':'','^Nancy.*$':'Nancy','^Celta.*$':'Celta de Vigo',' C\.F\.':'','^SM ':'','^En Avant( de | )':'','^SV ':'','^FC Ingolstadt 04$':'Ingolstadt','^FC Lorient$':'Lorient','^Stade de ':'','^Stade ':'','^FC Nantes$':'Nantes',' Alsace$':'','^Malherbe ':'','^SD Huesca$':'Huesca','^(FC |)Girondins de ':'','^Football Club de ':'','^FC Metz$':'Metz','^LOSC ':'',' Olympique$':'',' Foot(| \d{2})$':'','^Paris$':'Paris Saint Germain','^Paris Saint-Germain$':'Paris Saint Germain',' 07$':'','^SC Bastia$':'Bastia','^Hertha.*Berlin':'Hertha BSC','^SC Paderborn$':'Paderborn','^FC Nurnberg$':'Nurnberg','^FC Schalke 04$':'Schalke 04','^Deportivo$':'Deportivo La Coruna','^Deportivo de La Coruna$':'Deportivo La Coruna','^Chievo Verona$':'Chievo','Brestois 29$':'Brest','^Olympique de Marseille$':'Olympique Marseille',' HSC$':'',' Herault SC$':'','^La Spezia$':'Spezia','^Rennais$':'Rennes','^R. Sporting$':'Sporting Gijon','^Real Sporting de Gijon$':'Sporting Gijon','^Athletic Club de Bilbao$':'Athletic Club','^Spal$':'SPAL',"Borussia M'gladbach":'Borussia Monchengladbach','^Arminia Bielefeld$':'DSC Arminia Bielefeld','^R\. Valladolid$':'Real Valladolid','^Latium$':'Lazio','^Amiens SC Football$':'Amiens SC','^Salerno$':'Salernitana','^Bergamo Calcio$':'Atalanta','^TSG 1899 Hoffenheim$':'TSG Hoffenheim','^D\. Alaves':'Deportivo Alaves','^Bournemouth$':'AFC Bournemouth','^FC Bayern Munich$':'FC Bayern Munchen','^Bayern Munchen$':'FC Bayern Munchen','Sport-Club Freiburg':'SC Freiburg','^VfL Bochum$':'VfL Bochum 1848','^Real Betis Balompie$':'Real Betis','^Atletico de Madrid$':'Atletico Madrid'}
    for old, new in clubs_name_replace_list.items():
        series = series.str.replace(old, new, regex=True)
    return series

In [3]:
def replacePositions(series):
    positions_replace_list = {'^AM$':'M','^AM-L$':'M','^AM-R$':'M','^C$':'M','^CD$':'D','^CD-L$':'D','^CD-R$':'D','^CF-L$':'A','^CF-R$':'A','^CM$':'M','^CM-L$':'M','^CM-R$':'M','^LB$':'D','^LF$':'A','^LM$':'S','^RB$':'D','^RF$':'A','^RM$':'M','^SW$':'D', '^CAM$':'M','^CB$':'D','^CDM$':'M','^CF$':'A','^CM$':'M','^GK$':'G','^LB$':'D','^LM$':'M','^LW$':'M','^LWB$':'D','^RB$':'D','^RM$':'M','^RW$':'M','^RWB$':'M','^ST$':'A', 'LF':'A', 'RF':'A', 'RWM':'M', 'LWM':'M'}
    for old, new in positions_replace_list.items():
        series = series.str.replace(old, new, regex=True)
    return series

In [4]:
def convertNameToEnglish(name):
    name.replace('-', ' ')
    if pd.isna(name):
        return name
    else:
        return unidecode(name)

In [6]:
def getLineupsDataframe(df):
    rows = len(df)
    lineups_list = []
    for row in range(rows):
        lineup = df['lineup'].iloc[row]
        lineups_list += ast.literal_eval(lineup)

    lineups_df = pd.DataFrame(lineups_list)
    return lineups_df

Clean lineups data

In [5]:
# Import data from csv
leagues_more = pd.read_csv('../../Data/Data Collection/leagues_more.csv', low_memory=False)
leagues_static = pd.read_csv('../../Data/Data Collection/leagues_static.csv', low_memory=False)

In [7]:
# Create lineups DataFrame
lineups_df = getLineupsDataframe(leagues_more)
lineups_df = lineups_df[['team_id', 'fixture_id', 'player_id', 'player_name', 'number', 'position']]

In [22]:
# Create lineups DataFrame
lineups_df = getLineupsDataframe(leagues_more)
lineups_df = lineups_df[['team_id', 'fixture_id', 'player_id', 'player_name', 'number', 'position']]

# Drop rows with player_id is null
lineups_df = lineups_df.dropna(subset=['player_id'])

# Change column with float type to int
lineups_df['player_id'] = lineups_df['player_id'].astype(int)
lineups_df['number'] = pd.to_numeric(lineups_df['number'], errors='coerce')
lineups_df = lineups_df.dropna(subset=['number']).astype({'number': int}) 

# Drop rows with incomplete lineups data
grouped_df = lineups_df.groupby(['fixture_id']).count().reset_index()
incomplete_lineups = grouped_df[grouped_df['team_id'] != 22]['fixture_id']
lineups_df = lineups_df[~lineups_df['fixture_id'].isin(incomplete_lineups)]

# Merge lineups_df with leagues_static based on fixture_id
lineups_df = pd.merge(lineups_df, leagues_static[['id', 'time_starting_at_date', 'localTeam_id', 'localTeam_name', 'visitorTeam_id', 'visitorTeam_name']], left_on='fixture_id', right_on='id', how='left')
lineups_df = lineups_df.drop(columns=['id'])

# Rename time_starting_at_date and change to Date Format
lineups_df = lineups_df.rename(columns={'time_starting_at_date': 'date'})
lineups_df['date'] = pd.to_datetime(lineups_df['date'], format="%Y-%m-%d")

# Drop rows too old (year <= 2015)
lineups_df = lineups_df[lineups_df['date'].dt.year > 2015]

# Create club column and drop useless columns
lineups_df['club'] = np.where(lineups_df['team_id'] == lineups_df['localTeam_id'], lineups_df['localTeam_name'], lineups_df['visitorTeam_name'])
lineups_df['club'] = np.where(lineups_df['team_id'] != lineups_df['localTeam_id'], lineups_df['visitorTeam_name'], lineups_df['club'])
lineups_df = lineups_df.drop(columns=['localTeam_id', 'localTeam_name', 'visitorTeam_id', 'visitorTeam_name'])
lineups_df = lineups_df[~lineups_df['club'].isna()]

# Convert player_name column and club column to English format
lineups_df['player_name'] = lineups_df['player_name'].apply(convertNameToEnglish)
lineups_df['club'] = lineups_df['club'].apply(convertNameToEnglish)

# Replace club column and position column
lineups_df['club'] = replaceClubsName(lineups_df['club'])
lineups_df['position'] = replacePositions(lineups_df['position'])

# Create firstname column and lastname column
lineups_df.loc[:, 'lastname'] = lineups_df.loc[:, 'player_name'].str.split().str[-1].str.split('.').str[-1]
lineups_df.loc[:, 'firstname'] = lineups_df.loc[:, 'player_name'].apply(lambda x: x.split()[0] if '.' not in x.split()[0] else '')

Clean matches data

In [9]:
# Creat matches DataFrame
matches_df = leagues_static[['id', 'time_starting_at_date', 'localteam_id', 'localTeam_name', 'scores_localteam_score', 'localTeam_logo_path', 'visitorteam_id', 'visitorTeam_name', 'scores_visitorteam_score', 'visitorTeam_logo_path']]
fixtures_list = lineups_df['fixture_id'].unique().tolist()
matches_df = matches_df[matches_df['id'].isin(fixtures_list)]
matches_df = matches_df.rename(columns={'id' : 'fixture_id', 'time_starting_at_date': 'date', 'localteam_id': 'home_id', 'localTeam_name': 'home_name', 'localTeam_logo_path': 'home_logo', 'scores_localteam_score': 'home_scored', 'visitorteam_id': 'away_id', 'visitorTeam_name': 'away_name', 'visitorTeam_logo_path': 'away_logo', 'scores_visitorteam_score': 'away_scored'})

# Convert date to Date Format
matches_df['date'] = pd.to_datetime(matches_df['date'], format="%d/%m/%Y")

In [10]:
matches_df.to_csv('../../Data/Data Preparation/matches_final.csv', index=False)

Clean players attributes data

In [ ]:
# Import data from csv
players_df = pd.read_csv('../../Data/Data Collection/players_weekly_complete.csv', low_memory=False)
players_df = players_df[(~players_df['Club'].isna()) & (~players_df['Marking'].isna())]
players_df = players_df[['ObservationDate', 'Name', 'FavoritePosition', 'Height', 'Weight', 'Club', 'BallControl', 'Marking', 'Dribbling', 'SlideTackle', 'StandTackle', 'Aggression', 'Reactions', 'Att.Position', 'Interceptions', 'Vision', 'Crossing', 'ShortPass', 'LongPass', 'Acceleration', 'Stamina', 'Strength', 'Balance', 'SprintSpeed', 'Agility', 'Jumping', 'Heading', 'ShotPower', 'Finishing', 'LongShots', 'Curve',  'FKAcc.', 'Penalties', 'Volleys', 'GKPositioning', 'GKDiving', 'GKHandling', 'GKKicking', 'GKReflexes', 'Overall', 'Potential']]
# Change ObservationDate to Date Format
players_df['ObservationDate'] = players_df['ObservationDate'].str.replace('Sept', 'Sep')
players_df['ObservationDate'] = players_df['ObservationDate'].str.replace('April', 'Apr.')
players_df['ObservationDate'] = players_df['ObservationDate'].str.replace('June', 'Jun.')
players_df['ObservationDate'] = players_df['ObservationDate'].str.replace('March', 'Mar.')
players_df['ObservationDate'] = players_df['ObservationDate'].str.replace('May', 'May.')
players_df['ObservationDate'] = players_df['ObservationDate'].str.replace('July', 'Jul.')
players_df['ObservationDate'] = players_df['ObservationDate'].map(lambda x: datetime.strptime(x, '%b. %d, %Y'))
# Drop rows too old (year <= 2012)
players_df = players_df[players_df['ObservationDate'].dt.year > 2012]

# Convert Name column and Club column to English format
players_df['Name'] = players_df['Name'].apply(convertNameToEnglish)
players_df['Club'] = players_df['Club'].apply(convertNameToEnglish)

# Replace club column and position column
players_df['Club'] = replaceClubsName(players_df['Club'])
players_df['FavoritePosition'] = replacePositions(players_df['FavoritePosition'])

# Drop duplicates rows
players_df = players_df.drop_duplicates()

# Convert float columns to int
float_columns = players_df.select_dtypes(include='float64').columns
players_df[float_columns] = players_df[float_columns].astype(int)

# Clean all values of attributes' columns (keeping only last 2 digits)
for i in players_df.loc[:, 'BallControl':].columns.tolist(): 
    players_df.loc[:, i] = players_df.loc[:, i].astype(str).str[-2:].astype(int)

Merge Players data and Lineups Data

In [122]:
def findAcceptablePlayers(lineup_row, fifa_df):
    # Filtering by club
    same_club_mask = fifa_df.loc[fifa_df['Club'] == lineup_row.club]

    # First, try matching by full player name
    temporary_mask = same_club_mask.loc[(same_club_mask['Name'] == lineup_row.player_name)]
    
    # If no match, try matching by last name and first name
    if temporary_mask.shape[0] == 0: 
        temporary_mask = same_club_mask.loc[(same_club_mask['Name'].str.contains(lineup_row.lastname)) & 
                                            (same_club_mask['Name'].str.contains(lineup_row.firstname)), :]
        
        # If still no match, try matching by either last name or first name
        if temporary_mask.shape[0] == 0: 
            temporary_mask = same_club_mask.loc[((same_club_mask['Name'].str.contains(lineup_row.lastname)) | 
                                                (same_club_mask['Name'].str.contains(lineup_row.firstname))), :]
            
            # If no match, try matching by favorite position
            if temporary_mask.shape[0] == 0: 
                temporary_mask = same_club_mask.loc[(same_club_mask['FavoritePosition'] == lineup_row.position), :]

    return temporary_mask


In [124]:
def getCloserDate(line_row, mask_data):
    # Assuming that both line_row and mask_data have a column named 'date' representing the date
    line_date = line_row.date
    
    # Calculate the absolute time difference in days between line_date and all dates in mask_data
    time_diff = (mask_data['ObservationDate'] - line_date).abs()

    # Find the index of the minimum time difference
    closest_index = time_diff.idxmin()

    # Get the row data from the closest observation
    closest_row = mask_data.loc[closest_index].to_dict()

    return closest_row

In [174]:
# Create an empty list to store new rows
new_rows = []

for row in lineups_df.itertuples(index=False):
    # Find acceptable players based on the current lineup row
    mask_df = findAcceptablePlayers(row, players_df)
    
    # Get additional information based on closer date if there are acceptable players
    dict_app = getCloserDate(line_row=row, mask_data=mask_df) if mask_df.shape[0] != 0 else {}
    
    # Convert the current row to a dictionary and update it with additional information
    new_row = row._asdict()
    new_row.update(dict_app)
    
    # Append the updated row to the list
    new_rows.append(new_row)

# Create a DataFrame using the list of new rows
new_df = pd.DataFrame(new_rows)


In [134]:
new_df

,team_id,fixture_id,player_id,player_name,number,position,date,club,lastname,firstname,ObservationDate,Name,FavoritePosition,Height,Weight,Club,BallControl,Marking,Dribbling,SlideTackle,StandTackle,Aggression,Reactions,Att.Position,Interceptions,Vision,Crossing,ShortPass,LongPass,Acceleration,Stamina,Strength,Balance,SprintSpeed,Agility,Jumping,Heading,ShotPower,Finishing,LongShots,Curve,FKAcc.,Penalties,Volleys,GKPositioning,GKDiving,GKHandling,GKKicking,GKReflexes,Overall,Potential
0,597,11937723,27850,Stanislav Lobotka,68,M,2020-06-28,Napoli,Lobotka,Stanislav,2020-07-01,Stanislav Lobotka,M,170,64,Napoli,82,69,77,68,76,57,74,67,74,76,68,84,77,66,85,59,81,63,76,79,45,62,64,68,68,55,56,43,13,10,7,16,14,79,84
1,366,99380,32545,Marco Fabian,10,M,2017-05-20,Eintracht Frankfurt,Fabian,Marco,2017-05-22,Marco Fabian,M,168,65,Eintracht Frankfurt,81,20,80,39,44,64,75,75,34,77,74,76,73,76,75,66,84,69,90,73,61,80,76,77,76,78,74,75,15,14,11,8,13,78,79
2,2642,11886348,35062,Sven Michel,11,A,2020-06-27,Paderborn,Michel,Sven,2020-06-24,Sven Michel,A,179,79,Paderborn,69,28,69,22,25,68,72,74,20,58,63,66,53,79,77,68,73,76,75,73,53,76,74,65,66,59,65,68,7,11,11,11,11,71,71
3,8,11867569,4125,Mohamed Salah,11,A,2020-07-11,Liverpool,Salah,Mohamed,2020-07-08,Mohamed Salah,M,175,71,Liverpool,89,38,90,41,43,63,92,91,55,84,79,84,75,94,85,75,91,92,91,69,59,80,90,84,83,69,81,79,11,14,14,9,14,90,90
4,1,10332923,129454,Felipe Anderson,8,M,2018-12-15,West Ham United,Anderson,Felipe,2018-12-13,Felipe Anderson,M,175,70,West Ham United,88,47,90,54,60,67,74,73,69,79,79,80,74,93,72,53,88,89,89,38,44,83,71,84,71,69,53,68,5,12,10,9,7,83,85


In [175]:
new_df.to_csv('../../Data/Data Preparation/merged_lineups_final.csv', index=False)